<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2024/blob/main/Unidad_1/04_Algoritmos_en_graficas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

En esta sesión veremos algunos algoritmos relacionados con gráficas

# Encontrar hojas

Dado un árbol, encuentra sus hojas. Recuerda que las hojas son aquellos vértices que tienen un único vecino, es decir, tienen grado 1.

In [2]:
G = nx.random_tree(10)
nx.draw(G, with_labels=True, font_weight='bold')

In [3]:
hojas = []
for v in G.nodes:
    if G.degree[v] == 1:
        hojas.append(v)
print(hojas)

Como las hojas tienen un único vecino, su grado es $1$, por lo que podemos aprovechar el atributo `degree` de la gráfica.

# Depth-first search (DFS)

Este algoritmo nos permite recorrer la gráfica por profundidad, es decir, dado un vértice inicial $v$, explora la gráfica la mayor distancia posible a lo largo de cada rama antes de regresar.

El recorrido de este algoritmo nos da un árbol, comúnmente llamado árbol de DFS, que suele ser muy útil.

Escribimos ahora nuestro algoritmo; lo definimos de manera recursiva:

In [1]:
def dfs(g, v, visited=None, dfs_path=None):
    if visited is None:
        visited = [False] * len(g.nodes)

    if dfs_path is None:
        dfs_path = []

    visited[v] = True

    dfs_path.append(v) # Aquí puede ir cualquier otra cosa que querramos hacer con cada vértice de la gráfica
    
    for u in g.adj[v]:
        if not visited[u]:
            dfs(g, u, visited, dfs_path)
    return dfs_path

Generamos una gráfica aleatoria para probarlo:

In [ ]:
G = nx.gnp_random_graph(8, 1/3)
nx.draw(G, with_labels=True, font_weight='bold')

In [ ]:
dfs(G, 0)

También podemos implementar el algoritmo de manera iterativa:

In [ ]:
def dfs_iterativo(g, v):


  dfs_path = []
  visitados = []
  pila = [v]

  while pila != []:
    v_actual = pila.pop()
    visitados.append(v_actual)
    dfs_path.append(v_actual)
    for u in g.adj[v_actual]:
      if u not in visitados:
        pila.append(u)
  return dfs_path

In [ ]:
dfs(G,0)

# Número de componentes conexas

El algoritmo BFS puede ayudarnos a saber cuántas componentes conexas tiene una gráfica G. Basta iterar sobre los vértices de G: Si $v$ no ha sido marcado, aplicamos BFS sobre v y aumentamos el número de componentes conexas en 1. Si el vértice ya ha sido marcado, continuamos con la iteración.

In [44]:
def num_componentes_conexas(G):
    visitados = [False] * len(G.nodes)
    num_componentes_conexas = 0
    for v in range(len(G.nodes)):
        if visitados[v] == False:
            num_componentes_conexas = num_componentes_conexas + 1
            visitados[v] = True
            dfs(G, v, visitados)
    return num_componentes_conexas

Probemos nuestro código en la siguiente gráfica:

In [55]:
G = nx.gnp_random_graph(15, 1/5, seed=42)
#G.add_node(15)
#G.add_edge(15,12)
nx.draw(G, with_labels=True, font_weight='bold')

In [47]:
num_componentes_conexas(G)

## Altura de un árbol

Encontremos la altura de un árbol usando DFS; el procedimiento es bastante sencillo: todo lo que tenemos que hacer es calcular la altura de cada uno de los vértices, guardando el valor más alto encontrado en cada llamada recursiva:

In [5]:
def tree_height(
    g, # Gráfica a analizar
    v, # Vértice inicial
    heights = None, # Lista de alturas
    max_height = None # Altura máxima
):
    if heights is None:
        heights = [-1] * len(g.nodes)
        heights[v] = 0

    if max_height is None:
        max_height = [0] # La hacemos una lista para guardarla de manera sencilla a lo largo de llamadas recursivas

    for u in g.adj[v]:
        if heights[u] == -1:
            heights[u] = heights[v] + 1
            if max_height[0] < heights[u]:
                max_height[0] = heights[u]
            tree_height(g, u, heights, max_height)
    return max_height[0], heights

Otra manera de hacer esto sin tener que estar actualizando la altura máxima es simplemente calculando todas las alturas, y después obteniendo el máximo de la lista:

In [ ]:
def tree_height_2(
    g, # Gráfica a analizar
    v, # Vértice inicial
    heights = None, # Lista de alturas
):
    if heights is None:
        heights = [-1] * len(g.nodes)
        heights[v] = 0

    for u in g.adj[v]:
        if heights[u] == -1:
            heights[u] = heights[v] + 1
            temp_height = tree_height_2(g, u, heights)
    return max(heights), heights

El problema con esto es que tiene que iterar sobre toda la lista de alturas para encontrar el máximo, lo cual puede consumir mucho tiempo extra cuando se tengan muchos nodos.

In [6]:
T = nx.random_tree(10)
nx.draw(T, with_labels=True, font_weight='bold', font_color="white")

In [9]:
m, h = tree_height(T, 0)
print(m)
print(h)

## Coloreado

El colorear una gráfica consiste en asignar un color a cada nodo, de modo que cada par de nodos adyacentes tengan colores distintos. En 1976, se probó que para cualquier gráfica planar (i.e., que las aristas no se cruzan) se necesitan a lo mucho [cuatro colores](https://en.wikipedia.org/wiki/Four_color_theorem), aunque el algoritmo que implementaremos usa más colores que este óptimo.

El procedimiento es como sigue: recorremos la gráfica utilizando DFS. Si un nodo no está coloreado, significa que no lo hemos visitado, así que lo visitamos y lo coloreamos con el primer color que no tengan sus vecinos. Repetimos esto hasta que todos los nodos estén coloreados.

Por simplicidad, identificamos los colores como números del 0 en adelante. Escribimos una función para encontrar el primer color no utilizado, dada una lista de los colores adyacentes (en otras palabras, el número más pequeño distinto de todos los números dados):

In [ ]:
def first_available_color(neighbor_colors):
    out = 0
    while True:
        if out not in neighbor_colors:
            return out
        out += 1

Luego, definimos una función para hacer DFS y colorear. Al final de la ejecución, regresa una lista con los índices de los colores de cada nodo:

In [ ]:
def graph_coloring(g, v, visited=None):
    if visited is None:
        visited = [-1] * len(g.nodes)

    neighbor_colors = [ visited[n] for n in g.adj[v] ]
    new_color = first_available_color(neighbor_colors)
    visited[v] = new_color

    for u in g.adj[v]:
        if visited[u] == -1:
            graph_coloring(g, u, visited)

    return visited

Sin embargo, esta función no es suficiente. Para entender por qué, consideremos la siguiente gráfica:

In [ ]:
G = nx.gnp_random_graph(15, 1/5, seed=42)
nx.draw(G, with_labels=True, font_weight='bold')

Podemos ver que tiene dos componentes conexas. Si iniciamos el recorrido DFS del nodo 12, lo que pasará es que no podrá visitar la otra componente conexa, y por lo tanto no actualizará los colores de sus vértices. Comprobando que esto pasa:

In [ ]:
graph_coloring(G, 12)

Para resolver esto, escribimos una función que itere sobre cada entrada de la lista `visited`, y si ve alguna que es `-1` (i.e., el vértice correspondiente no ha sido visitado), llama la función `graph_coloring`. De esta manera, todas las componentes conexas se visitarán:

In [ ]:
def super_graph_coloring(g, visited=None):
    if visited is None:
        visited = [-1] * len(g.nodes)

    for v, vis in enumerate(visited):
        if vis == -1:
            graph_coloring(g, v, visited)

    return visited

Repitiendo sobre la gráfica anterior:

In [ ]:
node_colors = super_graph_coloring(G)
node_colors

Ahora podemos ver que todos los vértices fueron visitados y coloreados apropiadamente.

Como bonus, mapeamos los índices a nombres de colores reales, y los utilizamos para graficar la gráfica:

In [ ]:
color_names = ['red', 'green', 'blue', 'orange']
color_map = [None] * len(node_colors)
for i, c in enumerate(node_colors):
    color_map[i] = color_names[c]
color_map

In [ ]:
nx.draw(G, with_labels=True, font_weight='bold', node_color=color_map)

## Ciclos

¿Cómo detectar si una gráfica tiene algún ciclo? Para esto, recorremos la gráfica utilizando DFS, y en cada paso, vamos guardando el *padre* del vértice, i.e., el vértice en el que estábamos en el paso anterior. Si en algún momento estamos en un vértice que tenga un vértice adyacente que ya fue visitado, y que es distinto de su padre, entonces sabemos que existe un ciclo.

In [ ]:
def dfs_cycle(g, v, parents=None, cycle=None):
    if cycle is None:
        cycle = [False]

    if parents is None:
        parents = [None] * len(g.nodes)
        parents[v] = v

    for u in g.adj[v]:
        if parents[u] is None:
            parents[u] = v
            dfs_cycle(g, u, parents, cycle)
        else:
            if parents[v] != u:
                cycle[0] = True

    return cycle[0]

Al igual que en el caso del coloreado, tenemos que tomar en cuenta la posibilidad de que existan distintas componentes conexas:

In [ ]:
def has_cycle(g):
    parents = [None] * len(g.nodes)
    cycle = False
    for v in g.nodes:
        if parents[v] is None:
            parents[v] = v
            component_cycle = dfs_cycle(g, v, parents)
            cycle = cycle | component_cycle
    return cycle

Generamos una gráfica para probar nuestro algoritmo:

In [ ]:
G = nx.gnp_random_graph(10, 1/4, seed=42)
nx.draw(G, with_labels=True, font_weight='bold')

In [ ]:
has_cycle(G)

Luego, generamos un árbol:

In [ ]:
G = nx.random_tree(10)
nx.draw(G, with_labels=True, font_weight="bold")

Comprobando que no tiene ciclos:

In [ ]:
has_cycle(G)

# Breadth-first search

Este algoritmos nos permite, a partir de cierto vértice recorrer la gráfica por anchura, es decir, irnos expandiendo a partir de un vértices inicial $v$, primero se visitan los vértices vecinos a $v$, después los vecinos de sus vecinos, y así sucesivamente.

Esto en particular nos permite conocer la distancia del vértice $v$ a cualquier otro vértice de la gráfica, lo cual no es posible con DFS. Además, el recorrido que se hace con este algoritmo nos da un árbol generador de la componente conexa en la que se encuentra el vértice de donde partimos.

Escribiremos un código por nuestra cuenta, por momento sin hacer uso de algoritmos de la librería NetworkX. Para esto, necesitaremos usar una estructura llamada **cola** (queue en inglés), cuya cualidad es que el primer objeto en ser insertado es el primero en ser eliminado.

In [64]:
from collections import deque

def bfs(g, v):
    visited = [False] * len(g.nodes)
    visited[v] = True

    q = deque()
    q.append(v)

    while q: # mientras q tenga elementos
        u = q.popleft()
        visited[u] = True
        
        print(u) # Pueden hacer lo que quieran
        
        for w in g.adj[u]:
            if not visited[w] and w not in q:
                q.append(w)

In [57]:
G = nx.gnp_random_graph(10, 1/5, seed=42)
nx.draw(G, with_labels=True, font_weight='bold', font_color="white")

In [65]:
bfs(G, 0)

## Caminos

¿Cómo podemos encontrar el camino más corto entre un par de vértices usando BFS?

Notemos que podemos hacer algunas ligeras modificaciones para lograr nuestro objetivo: en lugar de que `visited` guarde si ya vimmos al vértice o no, crearemos una lista `parents` que guarde al vértice 'padre' de cada vértice que se vaya visitando, es decir, si $u$ es vecino de $v$ y $u$ no ha sido visitado en el recorrido de nuestro BFS, haremos `parents[u] = v`, indicando que justo para pasar a $u$ pasamos por $v$ previamente.

Notemos que al tener guardados estos 'padres' de los vértices podemos reconstruir fácilmente un camino de longitud mínima de entre `u` y `v`, pues basta con ir caminando sobre los padres de los vértices, comenzando por `v`, hasta llegar a `u`.

In [ ]:
def bfs_parents(g, v):
    parents = [None] * len(g.nodes)
    parents[v] = v

    q = deque()
    q.append(v)

    while(q):
        u = q.popleft()
        for w in g.adj[u]:
            if parents[w] is None:
                parents[w] = u
                q.append(w)
    parents[v] = None
    return parents

Aplicándolo a la gráfica anterior, e imprimiendo los padres resultantes:

In [ ]:
par = bfs_parents(G, 0)
for i, p in enumerate(par):
    print(f"{i} es hijo de {p}")

Naturalmente, si cambiamos el vértice inicial, las relaciones padre-hijo cambian:

In [ ]:
par = bfs_parents(G, 7)
for i, p in enumerate(par):
    print(f"{i} es hijo de {p}")

Con esta lista de padres, podemos reconstruir un camino entre cualquier par de vértices. Empezamos en el vértice final, y vamos visitando su padre, luego el padre de su padre, y así sucesivamente hasta llegar al vértice inicial.

In [ ]:
def path(g, start, end):
    parents = bfs_parents(g, start)
    if parents[end] is None: # No están en la misma componente conexa
        return []
    else:
        out = [end]
        while parents[end] != start:
            end = parents[end]
            out.append(end)
        out.append(start)
    return out

Usándolo para encontrar el camino entre el nodo 0 y el 7:

In [ ]:
path(G, 0, 7)

Y entre el 4 y el 7 (que no se encuentran en la misma componente conexa):

In [ ]:
path(G, 4, 7)

# Árboles generadores

El árbol generador de una gráfica $G$ es un árbol que contiene todos los vértices de $G$. Podemos usar DFS (o BFS) para producirlo de la siguiente manera:

1. Caminamos toda la gráfica y obtenemos los padres de cada vértice.
2. Insertamos una arista entre hijo y su padre.

Al árbol resultante se le conoce como árbol DFS (BFS).

Dado que ya tenemos una función para generar los padres con BFS (`bfs_parents`), escribimos una para hacerlo usando DFS:

In [ ]:
def dfs_parents(g, v, parents=None):
    if parents is None:
        parents = [None] * len(g.nodes)
        parents[v] = v

    for u in g.adj[v]:
        if parents[u] is None:
            parents[u] = v
            dfs_parents(g, u, parents)
    return parents

Luego, escribimos una función para generar un árbol dada una lista de parentezco:

In [ ]:
def tree_from_parents(g, parents):
    tree = nx.Graph()
    tree.add_nodes_from(g.nodes)
    for child, parent in enumerate(parents):
        if child is not None and parent is not None and child != parent:
            tree.add_edge(child, parent)
    return tree

Generamos una gráfica de prueba:

In [ ]:
G = nx.gnp_random_graph(15, 1/5, seed=42)
nx.draw(G, with_labels=True, font_weight='bold')

Calculamos sus parentezcos con BFS y DFS:

In [ ]:
bfs_par = bfs_parents(G, 0)
dfs_par = dfs_parents(G, 0)

Generamos los árboles:

In [ ]:
bfs_tree = tree_from_parents(G, bfs_par)
dfs_tree = tree_from_parents(G, dfs_par)

Y graficamos:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
nx.draw(bfs_tree, ax=ax1, with_labels=True, font_weight="bold")
nx.draw(dfs_tree, ax=ax2, with_labels=True, font_weight="bold")

# Ejercicios

## Ejercicio 1

Escribe un algoritmo `num_connected(G)` que, dada una gráfica `G`, regrese el número de componentes conexas que tiene.

*Aquí va la descripción y explicación de tu algoritmo*

In [ ]:
# Aquí va tu código

## Ejercicio 2

Dada una matriz de 0s y 1s, donde los 0s representan agua y los 1s tierra, escribe un algoritmo de NetworkX que cuente el número de *islas* que hay. Una isla se define como un conjunto de celdas de tierra conectadas por alguno de sus cuatro lados. Deberás escribir una función para convertir la matriz a una gráfica que la represente adecuadamente.

Por ejemplo, consideremos la siguiente matriz:

In [ ]:
M = np.zeros((10, 10))
M[0, 0] = M[1, 0] = M[2, 0] = M[3, 0] = M[3, 1] = M[4, 1] = 1
M[5, 5] = 1
M[7, 0] = M[8, 0] = M[9, 0] = 1
M[6, 6] = 1

plt.imshow(M)

Tu función debe de regresar **4** islas (nota que las dos islas de la derecha están separadas, ya que solo se tocan diagonalmente).

*Aquí va la descripción y explicación de tu algoritmo*

In [ ]:
# Aquí va tu función